In [22]:
# -*- coding: utf-8 -*-
"""
Script Python para obter dados climáticos da OpenWeatherMap API,
realizar uma análise simplificada de risco de alagamento e
gerar um resumo com a Google Gemini API.

Este script demonstra a integração de APIs de clima e modelos de linguagem
para uma avaliação preliminar de risco, mas NÃO substitui sistemas
oficiais de alerta hidrológico.
"""

import requests # Importa a biblioteca para fazer requisições HTTP (chamadas de API)
import json     # Importa a biblioteca para trabalhar com dados JSON
from datetime import datetime # Importa datetime para manipular datas e horas
from collections import defaultdict # Importa defaultdict para agrupar dados de previsão por dia
import google.generativeai as genai # Importa a biblioteca oficial do Google Gemini
import sys      # Importa a biblioteca sys (usada aqui para verificar versão, embora não essencial para este código)

# --- Configurações Essenciais ---

# Chave da API OpenWeatherMap
# É altamente recomendado armazenar chaves de API de forma segura,
# como em variáveis de ambiente ou usando o gerenciador de segredos do Google Colab.
# Substitua 'Weather_API' pelo nome exato do seu segredo no Google Colab, se aplicável.
try:
    # Tenta carregar a chave do gerenciador de segredos do Google Colab
    from google.colab import userdata
    OPENWEATHER_API_KEY = userdata.get('Weather_API')
    print("Chave da API OpenWeatherMap carregada do Google Colab usando o nome 'Weather_API'.")
except ImportError:
    # Caso não esteja no ambiente Google Colab ou o segredo não seja encontrado,
    # usa um placeholder. Em produção, carregue de forma segura (ex: variáveis de ambiente).
    # Exemplo: import os; OPENWEATHER_API_KEY = os.getenv('OPENWEATHER_API_KEY')
    OPENWEATHER_API_KEY = "Weather_API" # !!! SUBSTITUA PELA SUA CHAVE REAL OU USE MÉTODO SEGURO !!!
    print("Não está no Google Colab ou segredo 'Weather_API' não encontrado. Usando chave placeholder ou variável de ambiente (se configurado).")


# Chave da API Google Gemini
# Similar à chave do OpenWeatherMap, armazene de forma segura.
# 'GOOGLE_API_KEY' é o nome padrão do segredo para a API Gemini no Google Colab.
try:
    # Tenta carregar a chave do gerenciador de segredos do Google Colab
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    print("Chave da API Google Gemini carregada do Google Colab.")
except ImportError:
    # Caso não esteja no ambiente Google Colab, usa um placeholder.
    # Em produção, carregue de forma segura (ex: variáveis de ambiente).
    # Exemplo: import os; GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
    GOOGLE_API_KEY = "GOOGLE_API_KEY" # !!! SUBSTITUA PELA SUA CHAVE REAL OU USE MÉTODO SEGURO !!!
    print("Não está no Google Colab. Usando chave placeholder ou variável de ambiente (se configurado).")

# Configura a API do Google Gemini com a chave carregada
if GOOGLE_API_KEY and GOOGLE_API_KEY != "GOOGLE_API_KEY":
    try:
        genai.configure(api_key=GOOGLE_API_KEY)
        print("API do Google Gemini configurada.")
    except Exception as e:
        print(f"Erro ao configurar a API do Google Gemini: {e}")
        print("Verifique se a chave da API Google Gemini é válida.")
else:
    print("Chave da API Google Gemini não configurada. A funcionalidade Gemini não estará disponível.")


# Cidade para buscar os dados climáticos.
# Use o formato "Nome da Cidade,Código do País" (ex: "London,UK", "Porto Alegre,BR").
# A API do OpenWeatherMap geralmente funciona melhor com nomes de cidades específicas.
CIDADE = "Porto Alegre,BR"

# --- Limiares Simplificados para Alerta de Alagamento ---
# Estes valores são ILUSTRATIVOS e devem ser AJUSTADOS com base no conhecimento
# local, dados históricos e características da área (tipo de solo, drenagem, etc.).
# NÃO SÃO VALORES UNIVERSAIS E NÃO GARANTEM PRECISÃO.

LIMIAR_CHUVA_1H_MM = 15  # Volume de chuva na última hora (em mm) para considerar risco
LIMIAR_CHUVA_3H_MM = 25  # Volume de chuva nas últimas 3 horas (em mm) para considerar risco

# Lista de descrições de tempo da OpenWeatherMap que podem indicar chuva intensa.
# A comparação é feita de forma case-insensitive.
DESCRICOES_ALERTA_CHUVA = [
    "heavy intensity rain",      # Chuva de intensidade pesada
    "very heavy rain",           # Chuva muito pesada
    "extreme rain",              # Chuva extrema
    "heavy intensity shower rain", # Chuva de banho de intensidade pesada
    "ragged shower rain",        # Chuva irregular
    "thunderstorm with heavy rain", # Tempestade com chuva pesada
    "thunderstorm with rain",    # Tempestade com chuva
]

# Lista de IDs de condição climática da OpenWeatherMap que indicam potencial para chuva forte/tempestade.
# Consulte a documentação da OpenWeatherMap para a lista completa de IDs:
# https://openweathermap.org/weather-conditions#weather-codes
CODIGOS_ID_ALERTA = [
    201, 202, # Tempestade com chuva/chuva forte
    211, 212, # Tempestade
    502, 503, 504, # Chuva forte/muito forte/extrema
    521, 522, # Chuva de banho irregular/forte
    531 # Tempestade irregular
]

# --- Funções ---

def obter_dados_climaticos(api_key: str, cidade: str) -> tuple[dict | None, dict | None]:
    """
    Busca dados climáticos atuais e de previsão da OpenWeatherMap API.

    Args:
        api_key: A chave da API OpenWeatherMap.
        cidade: O nome da cidade e código do país (ex: "Sao Paulo,BR").

    Returns:
        Uma tupla contendo dois dicionários: (dados_atuais, dados_previsao).
        Retorna (None, None) em caso de erro na requisição ou decodificação JSON.
    """
    base_url = "http://api.openweathermap.org/data/2.5/"
    params = {
        'appid': api_key,
        'units': 'metric', # Unidades métricas (Celsius)
        'lang': 'pt_br',   # Idioma português do Brasil
        'q': cidade        # Nome da cidade e país
    }

    try:
        # --- Obter dados climáticos atuais ---
        url_atual = f"{base_url}weather"
        print(f"Buscando dados atuais para {cidade}...")
        resposta_atual = requests.get(url_atual, params=params)
        resposta_atual.raise_for_status()  # Lança exceção para status codes de erro (4xx ou 5xx)
        dados_atuais = resposta_atual.json()
        print("Dados atuais obtidos com sucesso.")

        # --- Obter dados de previsão (5 dias / a cada 3 horas) ---
        url_previsao = f"{base_url}forecast"
        print(f"Buscando previsão para {cidade}...")
        resposta_previsao = requests.get(url_previsao, params=params)
        resposta_previsao.raise_for_status() # Lança exceção para status codes de erro
        dados_previsao = resposta_previsao.json()
        print("Dados de previsão obtidos com sucesso.")

        return dados_atuais, dados_previsao

    except requests.exceptions.HTTPError as errh:
        print(f"Erro HTTP ao obter dados climáticos: {errh}")
    except requests.exceptions.ConnectionError as errc:
        print(f"Erro de Conexão ao obter dados climáticos: {errc}")
    except requests.exceptions.Timeout as errt:
        print(f"Erro de Timeout ao obter dados climáticos: {errt}")
    except requests.exceptions.RequestException as err:
        print(f"Ocorreu um erro na requisição HTTP: {err}")
    except json.JSONDecodeError:
        print("Erro ao decodificar a resposta JSON. A API pode estar fora ou a chave é inválida.")
    except Exception as e:
        print(f"Ocorreu um erro inesperado ao obter dados climáticos: {e}")

    return None, None # Retorna None em caso de qualquer erro

def analisar_risco_alagamento(dados_atuais: dict | None, dados_previsao: dict | None) -> bool:
    """
    Analisa os dados climáticos obtidos para uma previsão SIMPLIFICADA de risco de alagamento.

    Esta análise é baseada APENAS nos limiares de chuva e descrições do tempo
    e NÃO considera outros fatores cruciais para alagamentos reais.

    Args:
        dados_atuais: Dicionário com os dados climáticos atuais da OpenWeatherMap.
        dados_previsao: Dicionário com os dados de previsão da OpenWeatherMap.

    Returns:
        True se algum indicador de risco for identificado com base nos critérios simplificados,
        False caso contrário.
    """
    if not dados_atuais:
        print("Não foi possível obter dados atuais para análise de risco.")
        return False # Não há dados para analisar

    print("\n--- Análise de Risco de Alagamento (Simplificada) ---")
    risco_identificado = False # Flag para indicar se algum risco foi encontrado

    # --- 1. Analisar Chuva Atual ---
    print("\nAnalisando condições atuais:")
    chuva_1h_atual = 0
    chuva_3h_atual = 0

    if 'rain' in dados_atuais:
        # Verifica se há dados de chuva na última hora
        if '1h' in dados_atuais['rain']:
            chuva_1h_atual = dados_atuais['rain']['1h']
            print(f"  Chuva na última hora: {chuva_1h_atual} mm")
            # Compara com o limiar de alerta para 1 hora
            if chuva_1h_atual > LIMIAR_CHUVA_1H_MM:
                print(f"  ALERTA: Volume de chuva na última hora ({chuva_1h_atual}mm) excede o limiar de {LIMIAR_CHUVA_1H_MM}mm.")
                risco_identificado = True
        # Verifica se há dados de chuva nas últimas 3 horas (menos comum em dados 'weather')
        if '3h' in dados_atuais['rain']:
            chuva_3h_atual = dados_atuais['rain']['3h']
            print(f"  Chuva nas últimas 3 horas: {chuva_3h_atual} mm")
            # Compara com o limiar de alerta para 3 horas
            if chuva_3h_atual > LIMIAR_CHUVA_3H_MM:
                print(f"  ALERTA: Volume de chuva nas últimas 3 horas ({chuva_3h_atual}mm) excede o limiar de {LIMIAR_CHUVA_3H_MM}mm.")
                risco_identificado = True
    else:
        print("  Sem registro de chuva nos dados atuais.")

    # --- 2. Analisar Descrição e ID do Tempo Atual ---
    if 'weather' in dados_atuais and dados_atuais['weather']:
        descricao_atual = dados_atuais['weather'][0]['description']
        id_condicao_atual = dados_atuais['weather'][0]['id']
        print(f"  Condição atual: {descricao_atual} (ID: {id_condicao_atual})")
        # Verifica se a descrição atual contém palavras-chave de alerta (case-insensitive)
        if any(alerta in descricao_atual.lower() for alerta in DESCRICOES_ALERTA_CHUVA):
            print(f"  ALERTA: Descrição do tempo atual ({descricao_atual}) sugere chuva intensa.")
            risco_identificado = True
        # Verifica se o ID da condição atual está na lista de IDs de alerta
        if id_condicao_atual in CODIGOS_ID_ALERTA:
            print(f"  ALERTA: ID da condição climática atual ({id_condicao_atual} - {descricao_atual}) indica potencial para chuva forte/tempestade.")
            risco_identificado = True


    # --- 3. Analisar Previsão de Chuva (próximos 5 dias) ---
    if dados_previsao and 'list' in dados_previsao:
        print("\nAnalisando previsão para os próximos 5 dias:")

        # Agrupar dados de previsão por dia para exibir Máx/Mín por dia
        previsao_por_dia = defaultdict(list)
        for item in dados_previsao['list']:
            # Obtém a data (ano-mês-dia) de cada item da previsão
            data_item = datetime.fromtimestamp(item['dt']).strftime('%Y-%m-%d')
            previsao_por_dia[data_item].append(item)

        # Iterar sobre os dias agrupados na previsão
        for data_dia, previsoes_dia in previsao_por_dia.items():
            # Calcular temperatura máxima e mínima para o dia
            temp_max_dia = -float('inf') # Inicializa com um valor muito baixo
            temp_min_dia = float('inf')  # Inicializa com um valor muito alto

            for item_previsao in previsoes_dia:
                 # A API forecast fornece temp_max e temp_min para cada intervalo de 3h
                if 'main' in item_previsao:
                    temp_max_dia = max(temp_max_dia, item_previsao['main'].get('temp_max', -float('inf')))
                    temp_min_dia = min(temp_min_dia, item_previsao['main'].get('temp_min', float('inf')))

            # Formata a data para exibição no cabeçalho
            data_formatada = datetime.strptime(data_dia, '%Y-%m-%d').strftime('%d/%m/%Y')

            # Imprime o cabeçalho do dia com as temperaturas máxima e mínima calculadas
            # Verifica se as temperaturas foram encontradas antes de imprimir
            if temp_max_dia != -float('inf') and temp_min_dia != float('inf'):
                 print(f"\n--- Previsão para {data_formatada} --- Máx {temp_max_dia:.1f}ºC - Mín {temp_min_dia:.1f}ºC")
            else:
                 print(f"\n--- Previsão para {data_formatada} --- Temperaturas Máx/Mín não disponíveis")


            # Iterar sobre os intervalos de 3 horas para este dia específico
            for item_previsao in previsoes_dia:
                dt_previsao = datetime.fromtimestamp(item_previsao['dt']) # Timestamp do intervalo
                descricao_previsao = item_previsao['weather'][0]['description'] # Descrição do tempo
                id_condicao_previsao = item_previsao['weather'][0]['id']       # ID da condição do tempo
                # Obtém o volume de chuva previsto para as próximas 3 horas, padrão 0 se não existir
                chuva_3h_previsao = item_previsao.get('rain', {}).get('3h', 0)

                # Formata a string de chuva para exibição
                chuva_str = f"{chuva_3h_previsao} mm (acumulada nas próximas 3 horas)" if chuva_3h_previsao > 0 else "N/A mm (sem previsão de chuva nas próximas 3 horas)"

                # Imprime os detalhes do intervalo de 3 horas
                print(f"  - {dt_previsao.strftime('%H:%M')}: {descricao_previsao}, Chuva: {chuva_str}")

                # Verifica condições de alerta para este intervalo de 3 horas
                # Usando metade do limiar de 3h para previsão como cautela (pode ser ajustado)
                limiar_previsao = LIMIAR_CHUVA_3H_MM / 2
                if any(alerta in descricao_previsao.lower() for alerta in DESCRICOES_ALERTA_CHUVA) or \
                   (chuva_3h_previsao and chuva_3h_previsao > limiar_previsao) or \
                   id_condicao_previsao in CODIGOS_ID_ALERTA:
                    print(f"  ALERTA PREVISÃO: Condição ({descricao_previsao}) ou volume de chuva ({chuva_3h_previsao}mm) previsto para {dt_previsao.strftime('%Y-%m-%d %H:%M')} pode ser intenso.")
                    risco_identificado = True # Marca risco se qualquer intervalo do dia tiver alerta

    else:
        print("Não foi possível obter dados de previsão detalhados para análise.")


    # --- Resumo Final da Análise ---
    if not risco_identificado:
        print("\nNenhum indicador de risco imediato de alagamento identificado com base nos critérios simplificados nos próximos 5 dias.")
    else:
        print("\nATENÇÃO: Potencial risco de alagamento identificado com base nos critérios simplificados em algum momento nos próximos 5 dias. Monitore as condições e fontes oficiais.")

    print("\nLembre-se: Esta é uma análise muito básica e NÃO substitui alertas oficiais ou análises hidrológicas detalhadas.")

    return risco_identificado # Retorna o resultado booleano da análise

def gerar_resumo_com_gemini(api_key: str | None, dados_atuais: dict | None, dados_previsao: dict | None, risco_identificado: bool) -> None:
    """
    Usa a API do Google Gemini para gerar um resumo do risco de alagamento
    com base nos dados e na análise simplificada.

    Args:
        api_key: A chave da API Google Gemini. Pode ser None ou placeholder se não configurada.
        dados_atuais: Dicionário com os dados climáticos atuais.
        dados_previsao: Dicionário com os dados de previsão.
        risco_identificado: Booleano indicando se a análise simplificada identificou risco.
    """
    # Verifica se a chave Gemini está configurada e não é o placeholder
    if not api_key or api_key == "GOOGLE_API_KEY":
        print("\nChave da API Google Gemini não configurada. Não é possível gerar resumo com Gemini.")
        return

    # Prepara o prompt (entrada de texto) para o modelo Gemini
    prompt = f"Com base nos seguintes dados climáticos e análise simplificada de risco de alagamento para a cidade de {CIDADE}, gere um breve resumo em português:\n\n"

    # Adiciona informações dos dados atuais ao prompt
    if dados_atuais:
        prompt += f"Condições atuais: Temperatura {dados_atuais.get('main', {}).get('temp', 'N/A')}°C, Umidade {dados_atuais.get('main', {}).get('humidity', 'N/A')}%, Descrição: {dados_atuais.get('weather', [{}])[0].get('description', 'N/A')}.\n"
        if 'rain' in dados_atuais:
             if '1h' in dados_atuais['rain']:
                 prompt += f"Chuva na última hora: {dados_atuais['rain']['1h']} mm.\n"
             if '3h' in dados_atuais['rain']:
                 prompt += f"Chuva nas últimas 3 horas: {dados_atuais['rain']['3h']} mm.\n"
        else:
            prompt += "Sem registro de chuva nos dados atuais.\n"
    else:
        prompt += "Dados atuais não disponíveis.\n"


    # Adiciona um resumo da previsão ao prompt (limitado às primeiras entradas para concisão)
    if dados_previsao and 'list' in dados_previsao:
        prompt += "\nPrevisão para as próximas horas (resumida, a cada 3 horas):\n"
        # Inclui as primeiras 5 entradas da previsão (aproximadamente 15 horas) no prompt
        for item_previsao in dados_previsao['list'][:5]:
            dt_previsao = datetime.fromtimestamp(item_previsao['dt'])
            descricao_previsao = item_previsao.get('weather', [{}])[0].get('description', 'N/A')
            chuva_3h_previsao = item_previsao.get('rain', {}).get('3h', 0)
            prompt += f"  - {dt_previsao.strftime('%Y-%m-%d %H:%M')}: {descricao_previsao}, Chuva (3h): {chuva_3h_previsao} mm.\n"
        if len(dados_previsao.get('list', [])) > 5:
             prompt += f"  ... (previsão completa para 5 dias disponível, analisada separadamente)\n"
    else:
        prompt += "\nNão foi possível obter dados de previsão detalhados.\n"

    # Adiciona o resultado da análise simplificada ao prompt
    prompt += f"\nAnálise de risco simplificada para os próximos 5 dias: {'Potencial risco de alagamento identificado em algum momento.' if risco_identificado else 'Nenhum indicador de risco imediato de alagamento identificado com base nos critérios simplificados.'}\n"

    # Instrução final para o Gemini gerar o resumo
    prompt += "\nPor favor, gere um resumo conciso (máximo 3-4 frases) sobre a situação atual e a previsão de curto/médio prazo (próximos 5 dias), incluindo a indicação de risco de alagamento se aplicável, e reforçando que a análise é baseada em critérios simplificados de chuva."

    try:
        # Cria uma instância do modelo generativo (usando um modelo comum para texto)
        # 'gemini-1.5-flash-latest' ou 'gemini-1.0-pro' são boas opções.
        model = genai.GenerativeModel('gemini-1.5-flash-latest')

        # Gera o conteúdo com base no prompt
        print("\nGerando resumo com Google Gemini...")
        response = model.generate_content(prompt)
        print("Resumo gerado.")

        # Imprime o resultado da geração
        print("\n--- Resumo Gerado pelo Google Gemini ---")
        # Verifica se a resposta contém texto antes de imprimir
        if response.text:
            print(response.text)
        else:
            print("Não foi possível gerar texto com o modelo Gemini. A resposta está vazia.")
            # Em caso de resposta vazia, pode ser útil imprimir a resposta completa para depuração
            # print("Resposta completa do Gemini:", response)

        print("---------------------------------------")

    except Exception as e:
        # Captura e imprime erros durante a chamada da API Gemini
        print(f"\nErro ao chamar a API do Google Gemini: {e}")
        print("Verifique se a chave da API Google Gemini está correta e se a biblioteca 'google-generativeai' está instalada.")


# --- Ponto de Entrada do Script ---
if __name__ == "__main__":
    # Verifica se a chave da API OpenWeatherMap foi carregada corretamente
    if OPENWEATHER_API_KEY is None or OPENWEATHER_API_KEY == "Weather_API":
        print("Erro: Chave da API OpenWeatherMap não configurada ou inválida.")
        print("Por favor, verifique se o segredo 'Weather_API' está configurado corretamente no Google Colab")
        print("ou substitua 'Weather_API' pela sua chave real se não estiver usando Colab.")
    else:
        # Executa o fluxo principal do script
        print(f"Iniciando análise climática para: {CIDADE}")

        # 1. Obter dados climáticos
        dados_atuais, dados_previsao = obter_dados_climaticos(OPENWEATHER_API_KEY, CIDADE)

        # 2. Exibir dados atuais (se disponíveis)
        if dados_atuais:
            print("\n--- Dados Atuais ---")
            # Usa .get() com valor padrão para evitar KeyError se algum campo estiver faltando
            print(f"Temperatura: {dados_atuais.get('main', {}).get('temp', 'N/A')}°C")
            print(f"Sensação Térmica: {dados_atuais.get('main', {}).get('feels_like', 'N/A')}°C")
            print(f"Umidade: {dados_atuais.get('main', {}).get('humidity', 'N/A')}%")
            # Verifica se a lista 'weather' existe e não está vazia antes de acessar o índice [0]
            if 'weather' in dados_atuais and dados_atuais['weather']:
                print(f"Descrição: {dados_atuais['weather'][0].get('description', 'N/A')}")
            # Verifica se há dados de chuva e imprime se existirem
            if 'rain' in dados_atuais and dados_atuais['rain']:
                if '1h' in dados_atuais['rain']:
                    print(f"Chuva (última 1 hora): {dados_atuais['rain']['1h']} mm")
                if '3h' in dados_atuais['rain']:
                    print(f"Chuva (últimas 3 horas): {dados_atuais['rain']['3h']} mm") # Geralmente não presente no 'current'
        else:
            print("\nNão foi possível exibir dados atuais.")


        # 3. Realizar a análise de risco de alagamento para os próximos 5 dias
        # A função retorna True se risco for identificado, False caso contrário.
        risco_identificado = analisar_risco_alagamento(dados_atuais, dados_previsao)

        # 4. Chamar a função para gerar o resumo com Gemini (se a chave estiver configurada)
        gerar_resumo_com_gemini(GOOGLE_API_KEY, dados_atuais, dados_previsao, risco_identificado)

    print("\nSua pesquisa foi finalizada.")



Chave da API OpenWeatherMap carregada do Google Colab usando o nome 'Weather_API'.
Chave da API Google Gemini carregada do Google Colab.
API do Google Gemini configurada.
Iniciando análise climática para: Porto Alegre,BR
Buscando dados atuais para Porto Alegre,BR...
Dados atuais obtidos com sucesso.
Buscando previsão para Porto Alegre,BR...
Dados de previsão obtidos com sucesso.

--- Dados Atuais ---
Temperatura: 17.36°C
Sensação Térmica: 17.45°C
Umidade: 88%
Descrição: nublado

--- Análise de Risco de Alagamento (Simplificada) ---

Analisando condições atuais:
  Sem registro de chuva nos dados atuais.
  Condição atual: nublado (ID: 804)

Analisando previsão para os próximos 5 dias:

--- Previsão para 17/05/2025 --- Máx 28.2ºC - Mín 17.4ºC
  - 06:00: nublado, Chuva: N/A mm (sem previsão de chuva nas próximas 3 horas)
  - 09:00: nublado, Chuva: N/A mm (sem previsão de chuva nas próximas 3 horas)
  - 12:00: nublado, Chuva: N/A mm (sem previsão de chuva nas próximas 3 horas)
  - 15:00: nu